## Tensorflow with example

Thanks to @edureka on youtube.com, I prepared the following notebook for a quick hands-on neural network model for a classification problem. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/Sonar.csv')

In [3]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,1


**Preparing the sample and labels:** 

Note the .values applied to df. For later use in tensorflow, X should be an array not a pandas data frame. 

In [4]:
X = df.drop('Class', axis = 1).values
y = df['Class']

In [5]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [6]:
y_o = one_hot_encode(y)

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y_o, test_size = 0.2, random_state = 0)

In [8]:
learning_rate = 0.3
training_epochs = 1500
cost_history = np.empty(shape=[1], dtype = float)
n_class = 2
model_path = "./dp_model/dp_solar_model"

We use a neural network with 6 layers (i.e. 4 hidden layers), with the following dimensions:

In [9]:
n_0 = X.shape[1] # this is equal to the number of features per sample
n_1 = 50 # size of hidden layer 1
n_2 = 50 # size of hidden layer 2
n_3 = 50 # size of hidden layer 3
n_4 = 50 # size of hidden layer 4
n_5 = len(np.unique(y)) # this is equal to the number of classes 

In [10]:
x_ = tf.placeholder(tf.float32, shape=[None, n_0], name = 'x_')
yreal_ = tf.placeholder(tf.float32, shape=[None, n_5], name = 'yreal_')

Overall, we have 6 layers: 0, 1, 2, 3, 4, 5. The first layer is denoted by $\boldsymbol{a}^{(0)}$ and the final layer is denoted by $a^{(L)}$ where $L = 5$. So, for each layer $l$, we have: $$\boldsymbol{a}^l = \mathrm{W}^l \boldsymbol{a}^{l - 1} + \boldsymbol{b}^l.$$

Even though for one sample it is convenient to write the linear map as above, with many samples we will rewrite the above forumlat as follows:
\begin{eqnarray}
\boldsymbol{a}^{(1)} = \boldsymbol{a}^{(l - 1)} \mathrm{W}^{(1)} + \boldsymbol{b}^{(1)}
\end{eqnarray}
where the dimenstion of $W$ is $n_0 \times n_1$.

Since $\boldsymbol{a}^{(0)}$ is indeed the input $(x)_{N \times n_0}$, where $N$ is the number of samples, the above formula reads as: 
\begin{eqnarray}
\boldsymbol{a}^{(1)} = \boldsymbol{x} \mathrm{W}^{(1)} + \boldsymbol{b}^{(1)}
\end{eqnarray}
Verify that the dimension of $\boldsymbol{a}^{(1)}$ is $N \times n_1$.

\begin{eqnarray}
\boldsymbol{z}^l = \boldsymbol{a}^{l - 1}  \mathrm{W}^l + \boldsymbol{b}^l
\end{eqnarray}


Therefore the dimensions of the weight matrices are as follows:

$\mathrm{W}^1$ is $n_0 \times n_1$

$\mathrm{W}^2$ is $n_1 \times n_2$

$\mathrm{W}^3$ is $n_2 \times n_3$

$\mathrm{W}^4$ is $n_3 \times n_4$

$\mathrm{W}^5$ is $n_4 \times n_5$

In [11]:
w = {
    '1' : tf.Variable(tf.truncated_normal([n_0, n_1])), 
    '2' : tf.Variable(tf.truncated_normal([n_1, n_2])),
    '3' : tf.Variable(tf.truncated_normal([n_2, n_3])),
    '4' : tf.Variable(tf.truncated_normal([n_3, n_4])),
    '5' : tf.Variable(tf.truncated_normal([n_4, n_5]))
}
b = {
    '1' : tf.Variable(tf.truncated_normal([n_1])),
    '2' : tf.Variable(tf.truncated_normal([n_2])),
    '3' : tf.Variable(tf.truncated_normal([n_3])),
    '4' : tf.Variable(tf.truncated_normal([n_4])),
    '5' : tf.Variable(tf.truncated_normal([n_5]))
}

The next step is to define the hypothesis function of the neural network. That is the formula for the output layer which is found by the sequence of $\boldsymbol{a}^l =  \boldsymbol{a}^{l - 1} \mathrm{W}^l + \boldsymbol{b}^l$

In [12]:
with tf.variable_scope('layer_1'):
    z_1 = tf.add(tf.matmul(x_, w['1']), b['1'])
    a_1 = tf.nn.sigmoid(z_1)

with tf.variable_scope('layer_2'):
    z_2 = tf.add(tf.matmul(a_1, w['2']), b['2'])
    a_2 = tf.nn.sigmoid(z_2)

with tf.variable_scope('layer_3'):
    z_3 = tf.add(tf.matmul(a_2, w['3']), b['3'])
    a_3 = tf.nn.sigmoid(z_3)

with tf.variable_scope('layer_4'):
    z_4 = tf.add(tf.matmul(a_3, w['4']), b['4'])
    a_4 = tf.nn.relu(z_4)

with tf.variable_scope('output_layer'):
    ypred_ = tf.add(tf.matmul(a_4, w['5']), b['5'])
#     ypred_ = tf.nn.relu(z_out)

In [13]:
cost_function_ = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=ypred_, labels=yreal_))

In [14]:
with tf.variable_scope('logging'):
    tf.summary.scalar('cost', cost_function_)
    summary = tf.summary.merge_all()

In [15]:
init = tf.global_variables_initializer()

In [16]:
training_step_ = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function_)

In [17]:
Xtrain.shape

(166, 60)

In [18]:
cost_history = []
accuracy_history = []

In [19]:
correct_prediction_ = tf.equal(tf.argmax(yreal_, 1), tf.argmax(ypred_, 1))

In [20]:
accuracy_ = tf.reduce_mean(tf.cast(correct_prediction_, tf.float32))

In [21]:
with tf.Session() as sess:
    sess.run(init)
    training_writer = tf.summary.FileWriter("../logs/training", sess.graph)
    for epoch in range(training_epochs):
        sess.run(training_step_, feed_dict={x_:Xtrain, yreal_:ytrain})
        cost, training_summary = sess.run([cost_function_, summary], feed_dict={x_:Xtrain, yreal_:ytrain})
        training_writer.add_summary(training_summary, epoch)
        accuracy = sess.run(accuracy_, feed_dict={x_:Xtrain, yreal_:ytrain})
        cost_history = np.append(cost_history, cost)
        accuracy_history = np.append(accuracy_history, accuracy_)
        print("epoch: ", epoch, " - cost", cost, "- accuracy ", accuracy)
# accuracy on the test set:
    sess.run(accuracy_, feed_dict={x_:Xtest, yreal_:ytest})
    ypredLabel_ = tf.argmax(ypred_, 1)
    ypredLabel = sess.run(ypredLabel_, feed_dict={x_:Xtest})

epoch:  0  - cost 92.6898 - accuracy  0.542169
epoch:  1  - cost 16.9627 - accuracy  0.457831
epoch:  2  - cost 3.58664 - accuracy  0.542169
epoch:  3  - cost 6.89485 - accuracy  0.457831
epoch:  4  - cost 4.21627 - accuracy  0.542169
epoch:  5  - cost 1.5246 - accuracy  0.542169
epoch:  6  - cost 1.34745 - accuracy  0.542169
epoch:  7  - cost 1.03257 - accuracy  0.542169
epoch:  8  - cost 0.690963 - accuracy  0.560241
epoch:  9  - cost 0.690577 - accuracy  0.548193
epoch:  10  - cost 0.697013 - accuracy  0.481928
epoch:  11  - cost 0.730368 - accuracy  0.542169
epoch:  12  - cost 0.77141 - accuracy  0.457831
epoch:  13  - cost 0.950362 - accuracy  0.542169
epoch:  14  - cost 0.687728 - accuracy  0.5
epoch:  15  - cost 0.719664 - accuracy  0.542169
epoch:  16  - cost 0.74927 - accuracy  0.457831
epoch:  17  - cost 0.897182 - accuracy  0.542169
epoch:  18  - cost 0.694661 - accuracy  0.457831
epoch:  19  - cost 0.751864 - accuracy  0.542169
epoch:  20  - cost 0.748449 - accuracy  0.4578

epoch:  173  - cost 0.526202 - accuracy  0.722892
epoch:  174  - cost 0.605543 - accuracy  0.686747
epoch:  175  - cost 0.795854 - accuracy  0.560241
epoch:  176  - cost 0.508777 - accuracy  0.759036
epoch:  177  - cost 0.484575 - accuracy  0.789157
epoch:  178  - cost 0.483222 - accuracy  0.789157
epoch:  179  - cost 0.469631 - accuracy  0.777108
epoch:  180  - cost 0.473406 - accuracy  0.771084
epoch:  181  - cost 0.472984 - accuracy  0.789157
epoch:  182  - cost 0.512514 - accuracy  0.728916
epoch:  183  - cost 0.522413 - accuracy  0.759036
epoch:  184  - cost 0.634095 - accuracy  0.668675
epoch:  185  - cost 0.451199 - accuracy  0.819277
epoch:  186  - cost 0.442119 - accuracy  0.813253
epoch:  187  - cost 0.437609 - accuracy  0.807229
epoch:  188  - cost 0.44786 - accuracy  0.795181
epoch:  189  - cost 0.475426 - accuracy  0.771084
epoch:  190  - cost 0.606613 - accuracy  0.698795
epoch:  191  - cost 0.561365 - accuracy  0.759036
epoch:  192  - cost 0.683008 - accuracy  0.608434
e

epoch:  353  - cost 0.567584 - accuracy  0.698795
epoch:  354  - cost 0.356503 - accuracy  0.855422
epoch:  355  - cost 0.343208 - accuracy  0.849398
epoch:  356  - cost 0.312174 - accuracy  0.861446
epoch:  357  - cost 0.343758 - accuracy  0.837349
epoch:  358  - cost 0.334293 - accuracy  0.831325
epoch:  359  - cost 0.451308 - accuracy  0.73494
epoch:  360  - cost 0.382126 - accuracy  0.807229
epoch:  361  - cost 0.478123 - accuracy  0.710843
epoch:  362  - cost 0.310507 - accuracy  0.879518
epoch:  363  - cost 0.290138 - accuracy  0.873494
epoch:  364  - cost 0.290184 - accuracy  0.86747
epoch:  365  - cost 0.317579 - accuracy  0.86747
epoch:  366  - cost 0.338795 - accuracy  0.819277
epoch:  367  - cost 0.49603 - accuracy  0.722892
epoch:  368  - cost 0.40692 - accuracy  0.783133
epoch:  369  - cost 0.485142 - accuracy  0.710843
epoch:  370  - cost 0.298928 - accuracy  0.891566
epoch:  371  - cost 0.279665 - accuracy  0.879518
epoch:  372  - cost 0.271639 - accuracy  0.891566
epoch

epoch:  538  - cost 0.238986 - accuracy  0.891566
epoch:  539  - cost 0.288829 - accuracy  0.861446
epoch:  540  - cost 0.655904 - accuracy  0.722892
epoch:  541  - cost 0.501631 - accuracy  0.740964
epoch:  542  - cost 0.431067 - accuracy  0.76506
epoch:  543  - cost 0.30272 - accuracy  0.849398
epoch:  544  - cost 0.288949 - accuracy  0.86747
epoch:  545  - cost 0.251747 - accuracy  0.903614
epoch:  546  - cost 0.258303 - accuracy  0.891566
epoch:  547  - cost 0.207331 - accuracy  0.933735
epoch:  548  - cost 0.21256 - accuracy  0.909639
epoch:  549  - cost 0.209835 - accuracy  0.921687
epoch:  550  - cost 0.251542 - accuracy  0.89759
epoch:  551  - cost 0.230378 - accuracy  0.909639
epoch:  552  - cost 0.355387 - accuracy  0.825301
epoch:  553  - cost 0.277288 - accuracy  0.879518
epoch:  554  - cost 0.350135 - accuracy  0.825301
epoch:  555  - cost 0.238426 - accuracy  0.903614
epoch:  556  - cost 0.283503 - accuracy  0.885542
epoch:  557  - cost 0.194493 - accuracy  0.927711
epoch

epoch:  713  - cost 0.860225 - accuracy  0.626506
epoch:  714  - cost 0.332958 - accuracy  0.86747
epoch:  715  - cost 0.300276 - accuracy  0.855422
epoch:  716  - cost 0.277297 - accuracy  0.873494
epoch:  717  - cost 0.261598 - accuracy  0.909639
epoch:  718  - cost 0.258835 - accuracy  0.903614
epoch:  719  - cost 0.265423 - accuracy  0.86747
epoch:  720  - cost 0.300468 - accuracy  0.849398
epoch:  721  - cost 0.201941 - accuracy  0.957831
epoch:  722  - cost 0.182873 - accuracy  0.957831
epoch:  723  - cost 0.163793 - accuracy  0.96988
epoch:  724  - cost 0.155623 - accuracy  0.96988
epoch:  725  - cost 0.149481 - accuracy  0.96988
epoch:  726  - cost 0.157216 - accuracy  0.951807
epoch:  727  - cost 0.14839 - accuracy  0.963855
epoch:  728  - cost 0.185223 - accuracy  0.933735
epoch:  729  - cost 0.156866 - accuracy  0.951807
epoch:  730  - cost 0.206835 - accuracy  0.921687
epoch:  731  - cost 0.173219 - accuracy  0.933735
epoch:  732  - cost 0.274923 - accuracy  0.879518
epoch:

epoch:  896  - cost 0.0188062 - accuracy  1.0
epoch:  897  - cost 0.0186295 - accuracy  1.0
epoch:  898  - cost 0.018482 - accuracy  1.0
epoch:  899  - cost 0.0183594 - accuracy  1.0
epoch:  900  - cost 0.0181598 - accuracy  1.0
epoch:  901  - cost 0.0181776 - accuracy  1.0
epoch:  902  - cost 0.0179808 - accuracy  1.0
epoch:  903  - cost 0.017799 - accuracy  1.0
epoch:  904  - cost 0.0175582 - accuracy  1.0
epoch:  905  - cost 0.0174682 - accuracy  1.0
epoch:  906  - cost 0.0172129 - accuracy  1.0
epoch:  907  - cost 0.0170629 - accuracy  1.0
epoch:  908  - cost 0.0168689 - accuracy  1.0
epoch:  909  - cost 0.0167833 - accuracy  1.0
epoch:  910  - cost 0.016565 - accuracy  1.0
epoch:  911  - cost 0.016428 - accuracy  1.0
epoch:  912  - cost 0.0162867 - accuracy  1.0
epoch:  913  - cost 0.016271 - accuracy  1.0
epoch:  914  - cost 0.0160414 - accuracy  1.0
epoch:  915  - cost 0.0159335 - accuracy  1.0
epoch:  916  - cost 0.0157896 - accuracy  1.0
epoch:  917  - cost 0.0156873 - accurac

epoch:  1076  - cost 0.00590261 - accuracy  1.0
epoch:  1077  - cost 0.00587881 - accuracy  1.0
epoch:  1078  - cost 0.00585286 - accuracy  1.0
epoch:  1079  - cost 0.00583121 - accuracy  1.0
epoch:  1080  - cost 0.00580384 - accuracy  1.0
epoch:  1081  - cost 0.00577256 - accuracy  1.0
epoch:  1082  - cost 0.00574921 - accuracy  1.0
epoch:  1083  - cost 0.00572776 - accuracy  1.0
epoch:  1084  - cost 0.00569481 - accuracy  1.0
epoch:  1085  - cost 0.00567884 - accuracy  1.0
epoch:  1086  - cost 0.00564533 - accuracy  1.0
epoch:  1087  - cost 0.00563024 - accuracy  1.0
epoch:  1088  - cost 0.00560138 - accuracy  1.0
epoch:  1089  - cost 0.00557777 - accuracy  1.0
epoch:  1090  - cost 0.00554948 - accuracy  1.0
epoch:  1091  - cost 0.00552142 - accuracy  1.0
epoch:  1092  - cost 0.00549834 - accuracy  1.0
epoch:  1093  - cost 0.00548748 - accuracy  1.0
epoch:  1094  - cost 0.00545527 - accuracy  1.0
epoch:  1095  - cost 0.00542983 - accuracy  1.0
epoch:  1096  - cost 0.0054105 - accurac

epoch:  1263  - cost 0.00302862 - accuracy  1.0
epoch:  1264  - cost 0.00302478 - accuracy  1.0
epoch:  1265  - cost 0.00301101 - accuracy  1.0
epoch:  1266  - cost 0.00300329 - accuracy  1.0
epoch:  1267  - cost 0.00299336 - accuracy  1.0
epoch:  1268  - cost 0.00298675 - accuracy  1.0
epoch:  1269  - cost 0.0029777 - accuracy  1.0
epoch:  1270  - cost 0.00297264 - accuracy  1.0
epoch:  1271  - cost 0.00296103 - accuracy  1.0
epoch:  1272  - cost 0.00295424 - accuracy  1.0
epoch:  1273  - cost 0.00294391 - accuracy  1.0
epoch:  1274  - cost 0.00293829 - accuracy  1.0
epoch:  1275  - cost 0.00292857 - accuracy  1.0
epoch:  1276  - cost 0.00292294 - accuracy  1.0
epoch:  1277  - cost 0.00291248 - accuracy  1.0
epoch:  1278  - cost 0.00290512 - accuracy  1.0
epoch:  1279  - cost 0.00289667 - accuracy  1.0
epoch:  1280  - cost 0.002892 - accuracy  1.0
epoch:  1281  - cost 0.00288039 - accuracy  1.0
epoch:  1282  - cost 0.00287366 - accuracy  1.0
epoch:  1283  - cost 0.00286787 - accuracy 

epoch:  1447  - cost 0.00196025 - accuracy  1.0
epoch:  1448  - cost 0.00195436 - accuracy  1.0
epoch:  1449  - cost 0.00195025 - accuracy  1.0
epoch:  1450  - cost 0.00194679 - accuracy  1.0
epoch:  1451  - cost 0.00194436 - accuracy  1.0
epoch:  1452  - cost 0.00193981 - accuracy  1.0
epoch:  1453  - cost 0.00193641 - accuracy  1.0
epoch:  1454  - cost 0.00193311 - accuracy  1.0
epoch:  1455  - cost 0.00193157 - accuracy  1.0
epoch:  1456  - cost 0.00192551 - accuracy  1.0
epoch:  1457  - cost 0.00192157 - accuracy  1.0
epoch:  1458  - cost 0.00191842 - accuracy  1.0
epoch:  1459  - cost 0.00191524 - accuracy  1.0
epoch:  1460  - cost 0.00191133 - accuracy  1.0
epoch:  1461  - cost 0.00190762 - accuracy  1.0
epoch:  1462  - cost 0.00190453 - accuracy  1.0
epoch:  1463  - cost 0.0019023 - accuracy  1.0
epoch:  1464  - cost 0.00189712 - accuracy  1.0
epoch:  1465  - cost 0.00189356 - accuracy  1.0
epoch:  1466  - cost 0.00189085 - accuracy  1.0
epoch:  1467  - cost 0.00188842 - accurac

Now we evaluate the accuracy of the trained model on the test set:

In [22]:
ytestLabel = np.argmax(ytest, 1)

In [23]:
pd.DataFrame(np.transpose([ypredLabel, ytestLabel]), columns=['Actual', 'Prediction'])

,Actual,Prediction
0,0,1
1,0,1
2,0,1
3,1,1
4,0,0
5,1,1
6,0,0
7,0,0
8,1,1
9,0,0
